In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 100% !important; }</style>"))

In [4]:
from datetime import datetime
import os

solutions_list = []
max_runs = 10
dct_with_header = False
save_results = True
_instance = 'CMT1'
_cluster_type = 'K-MEANS'

# Creación del archivo de texto plano con los resultados
now = datetime.now()
dt_string = now.strftime('%Y%m%d_%H-%M-%S')
aco_cluster = 'Restricted Ant + ' + _cluster_type # Titulo del archivo .txt
ant_type = 'RA' # FA o RA (FREE ANT - RESTRICTED ANT) para el titulo del archivo
cluster_type = _cluster_type.replace('-', '').upper()[0:4] # KMEN o KMED (K-MEANS - K-MEDOIDS) para el titulo del archivo
instance = _instance # Nombre de la carpeta
file_folder = 'instances_results/' + instance # Carpeta raiz de resultados
file_folder_img = 'instances_results/' + instance + '/img_results'
if not os.path.exists(file_folder): os.makedirs(file_folder)
if not os.path.exists(file_folder_img): os.makedirs(file_folder_img)
file_name = ant_type + '_' + cluster_type + '_' + dt_string + '.txt'

for run in range(max_runs): 
    %matplotlib inline
    from restricted_bwacs_model import RestrictedBWACS
    aco_model = RestrictedBWACS(
        # Nombre de la instancia
        instance = 'CVRPLIB/CMT/' + _instance,
        # Número máximo de nodos
        max_nodes = 999, 
        # Costo a minimizar
        ant_type = 'emvrp',
        # Tipo de clustering | kmedoids - kmeans |
        cluster_type = _cluster_type.replace('-', '').lower(),
        # Métrica de distancia | euclidian - manhattan |
        metric = 'euclidian',
        # Peso del vehículo | corresponde al producto entre la cápacidad máxima y un porcentaje = Q * 0.15
        tare = 0.15,
        # Número máximo de iteraciones
        max_iterations = 50,
        # Cántidad de hormigas por iteracion
        max_ants = 25,
        # Inicializar cada hormiga en un nodo disinto dentro de una lista de candidatos. De lo contrario cada hormiga comenzara su recorrido en el depot | 0 - 1 |
        start_ant_on_best_nodes = 1,
        # Valor de importancia de las feromonas / Mejor alpha: [1, 3]
        alpha = 3,
        # Valor de importancia de la distancia
        beta = 2,
        # Valor de importancia de la energia
        gamma = 3,
        # Valor de importancia del ahorro de distancia
        delta = 1,
        # Valor de importancia del ahorro de energia
        eta = 1,
        # Valor de importancia de la utilización de capacidad
        mi = 0,
        # Que tipo de estrategia de actualización de feromonas usar (1 / BestQuality) o Ant Weight Strategy | 0 - 1 |
        pheromone_updating_strategy = 1,
        # Actualización de feromonas por hormiga | 0 - 1 |
        local_ant_update_pheromones = 0,
        # Actualización de feromonas por la mejor de la iteración | 0 - 1 |
        best_iteration_update_pheromones = 1,
        # Actualización de feromonas por la mejor global | 0 - 1 |
        best_global_update_pheromones = 1,
        # Penalizar peor solución de la iteración | 0 - 1 |
        penalize_worst_solution = 1,
        # Mutar matriz de feromonas | 0 - 1 |
        mutate_pheromones_matrix = 1,
        # Ratio de evaporación de las feromonas | (1 - p) |
        p = 0.02,
        # Probabilidad de mutación de la matriz de feromonas
        Pm = 0.3, 
        # Intensidad de la mutación de la matriz de feromonas
        sigma = 2, 
        # Probabilidad de elegir el mejor nodo
        q0 = 0.3, 
        # Constante para calcular las funciones de costos entre otras | Q / BestTour |
        Q = 1,
        # El tipo de información heuristica a utilizar
        # 0 - Solo la información de distancias / Mejor beta: 5
        # 1 - Solo la información de energias / Mejor gamma: 5
        # 2 - Información de distancias X energias / Mejores parametros: 5 - 5
        # 3 - Solo información de ahorro de distancia | saving's algorithm | / Mejor delta: 10
        # 4 - Solo información de ahorro de energia | saving's algorithm | / Mejor eta: 10
        # 5 - Información de distancias X ahorro de distancia / Mejores parametros: 4 - 5
        # 6 - Información de energias X ahorro de energia / Mejores parametros: 5 - 5
        # 7 - Información de distancias X información energias X ahorro de distancia / Mejores parametros: 3 - 4 - 3
        # 8 - Información de distancias X información energias X ahorro de energias / Mejores parametros: 3 - 4 - 3
        # 9 - Información de distancias X información de energias X ahorro de distancia X información de capacidad / Mejores parametros: 2 - 4 - 4 - 2
        # 10 - Información de distancias X información de energias X ahorro de energia X información de capacidad / Mejores parametros: 2 - 4 - 4 - 2
        # 11 - Información de distancias X información de energias X ahorro de distancia X ahorro de energia X información de capacidad / Parametros: 2 - 4 - 3 - 4 - 1
        heuristic_type = 8,
        # Busqueda local a la solución individual de cada hormiga | 0 - 1 |
        ls_ant_solution = 0,
        # Busqueda local a la solución final | 0 - 1 |
        ls_final_solution = 0,
        # Output folder/name solution draw
        output_sol_img = file_folder_img + '/' + str(run + 1) + '__' + ant_type + '_' + cluster_type + '_' + dt_string + '.png'
    )
    
    solution_energy, solution_distance, solution_time = aco_model.solve()
    solutions_list.append((solution_energy, solution_distance, solution_time))

print(solutions_list)
print("Best Solution: " + str(min(solutions_list, key=lambda x: x[0])) + " on run: " + str(solutions_list.index(min(solutions_list, key=lambda x: x[0])) + 1))

if (save_results):
    file = open(file_folder + '/' + file_name, 'a')
    if dct_with_header: file.write('Resultados para la ' + instance + ' (' + aco_cluster + ') [energia, distancia, tiempo de ejecución]\n\n')
    for i, solution in enumerate(solutions_list):
        if dct_with_header: file.write(str(i + 1) + '| ' + str(solution) + '\n')
        else: 
            str_sol = str(solution).replace('(', '').replace(')', '').replace(',', '').replace('.', ',')
            file.write(str_sol + '\n')

    file.close()

• INICIALIZANDO ALGORITMO •
------------------------------

• PARAMETROS DE LA INSTANCIA: 

    > Nodos y demandas: {0: 0, 1: 7, 2: 30, 3: 16, 4: 9, 5: 21, 6: 15, 7: 19, 8: 23, 9: 11, 10: 5, 11: 19, 12: 29, 13: 23, 14: 21, 15: 10, 16: 15, 17: 3, 18: 41, 19: 9, 20: 28, 21: 8, 22: 8, 23: 16, 24: 10, 25: 28, 26: 7, 27: 15, 28: 14, 29: 6, 30: 19, 31: 11, 32: 12, 33: 23, 34: 26, 35: 17, 36: 6, 37: 9, 38: 15, 39: 14, 40: 7, 41: 27, 42: 13, 43: 11, 44: 16, 45: 10, 46: 5, 47: 25, 48: 17, 49: 18, 50: 10}
    > Demanda total: 777
    > Capacidad vehiculo: 160
    > K-Optimo: 5
    > Tightnessratio: 0.97125

• INICIANDO CLUSTERIZACIÓN: 

    > CLUSTER TYPE: KMEANS
    > ITERACIÓN 1
        - Centroides actuales: [array([57, 58]), array([17, 33]), array([10, 17]), array([61, 33]), array([ 7, 38])]
        - Nodos sin asignar: [ 7 30], [23 11]
        - Mejor costo anterior: inf
        - Costo total actual: 9739.5597891085

    > ITERACIÓN 2
        - Centroides actuales: [array([37., 69.]), array

e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\restricted_bwacs_model.py:445: RuntimeWarning: divide by zero encountered in true_divide
  self.np.divide(1, self.distances_matrix), self.beta)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\restricted_bwacs_model.py:447: RuntimeWarning: divide by zero encountered in true_divide
  self.np.divide(1, self.energies_matrix), self.gamma)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\restricted_bwacs_model.py:453: RuntimeWarning: invalid value encountered in multiply
  self.combination_matrix, _saving_energies_matrix)


        - Centroides actuales: [array([48.36363636, 57.72727273]), array([46.6, 36.5]), array([ 7., 38.]), array([45.77777778, 18.77777778]), array([ 5., 64.])]
        - Nodos sin asignar: [40], [27]
        - Mejor costo anterior: 8066.528567941032
        - Costo total actual: 10771.024281503887

    > ITERACIÓN 41
        - Centroides actuales: [array([48.18181818, 52.27272727]), array([46.33333333, 36.44444444]), array([10., 17.]), array([42. , 17.4]), array([20.4, 53.5])]
        - Nodos sin asignar: [30], [11]
        - Mejor costo anterior: 8066.528567941032
        - Costo total actual: 9760.871029384165

    > ITERACIÓN 42
        - Centroides actuales: [array([37., 69.]), array([48.6, 34.4]), array([19.88888889, 25.66666667]), array([35.8, 20.2]), array([19.2, 55.8])]
        - Nodos sin asignar: [], []
        - Mejor costo anterior: 8066.528567941032
        - Costo total actual: 9867.190655216931

    > Mejor Clusterización: [array([ 2, 16, 29, 11, 22, 32, 20,  1,  3, 35]

e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\restricted_bwacs_model.py:445: RuntimeWarning: divide by zero encountered in true_divide
  self.np.divide(1, self.distances_matrix), self.beta)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\restricted_bwacs_model.py:447: RuntimeWarning: divide by zero encountered in true_divide
  self.np.divide(1, self.energies_matrix), self.gamma)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\restricted_bwacs_model.py:453: RuntimeWarning: invalid value encountered in multiply
  self.combination_matrix, _saving_energies_matrix)


        - Centroides actuales: [array([43.9, 63.2]), array([62., 42.]), array([20. , 19.7]), array([49. , 22.3]), array([16.55555556, 48.44444444])]
        - Nodos sin asignar: [39], [7]
        - Mejor costo anterior: 8770.75270035537
        - Costo total actual: 10696.56634254118

    > ITERACIÓN 15
        - Centroides actuales: [array([41.16666667, 61.58333333]), array([47.77777778, 39.        ]), array([5., 6.]), array([46. , 22.2]), array([18.66666667, 49.        ])]
        - Nodos sin asignar: [46], [25]
        - Mejor costo anterior: 8770.75270035537
        - Costo total actual: 10035.503670792263

    > ITERACIÓN 16
        - Centroides actuales: [array([45.1, 62.9]), array([51.5, 40.5]), array([16.44444444, 18.66666667]), array([44. , 21.1]), array([25., 32.])]
        - Nodos sin asignar: [42], [11]
        - Mejor costo anterior: 8770.75270035537
        - Costo total actual: 10623.636803568075

    > ITERACIÓN 17
        - Centroides actuales: [array([41.41666667, 60.

...

        - Centroides actuales: [array([44.9, 59.3]), array([54.63636364, 37.09090909]), array([15.88888889, 20.55555556]), array([46., 10.]), array([15.33333333, 54.        ])]
        - Nodos sin asignar: [35], [6]
        - Mejor costo anterior: 8770.75270035537
        - Costo total actual: 9550.93543363689

    > ITERACIÓN 19
        - Centroides actuales: [array([63., 69.]), array([54.8, 40. ]), array([18.88888889, 22.33333333]), array([38.5, 18.3]), array([17.3, 50.5])]
        - Nodos sin asignar: [], []
        - Mejor costo anterior: 8770.75270035537
        - Costo total actual: 10439.904257073627

    > ITERACIÓN 20
        - Centroides actuales: [array([47.3, 61. ]), array([48.09090909, 35.36363636]), array([18.88888889, 24.22222222]), array([41.2, 19.9]), array([16.8, 53.4])]
        - Nodos sin asignar: [24 25], [28  7]
        - Mejor costo anterior: 8770.75270035537
        - Costo total actual: 8306.319175559167

    > ITERACIÓN 21
        - Centroides actuales: [array

e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\restricted_bwacs_model.py:445: RuntimeWarning: divide by zero encountered in true_divide
  self.np.divide(1, self.distances_matrix), self.beta)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\restricted_bwacs_model.py:447: RuntimeWarning: divide by zero encountered in true_divide
  self.np.divide(1, self.energies_matrix), self.gamma)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\restricted_bwacs_model.py:453: RuntimeWarning: invalid value encountered in multiply
  self.combination_matrix, _saving_energies_matrix)


        - Centroides actuales: [array([49.54545455, 48.63636364]), array([52.27272727, 29.54545455]), array([12.        , 27.71428571]), array([28.90909091, 26.54545455]), array([23. , 60.4])]
        - Nodos sin asignar: [27 35], [14  6]
        - Mejor costo anterior: 8306.319175559167
        - Costo total actual: 8066.528567941032

    > ITERACIÓN 30
        - Centroides actuales: [array([63., 69.]), array([52.45454545, 26.45454545]), array([12.        , 27.71428571]), array([27.9, 22.5]), array([21.7, 58.5])]
        - Nodos sin asignar: [32 10], [23 19]
        - Mejor costo anterior: 8066.528567941032
        - Costo total actual: 9162.6705524307

    > ITERACIÓN 31
        - Centroides actuales: [array([48.18181818, 53.81818182]), array([42., 41.]), array([12.        , 27.71428571]), array([31.3, 22.9]), array([23.18181818, 55.81818182])]
        - Nodos sin asignar: [], []
        - Mejor costo anterior: 8066.528567941032
        - Costo total actual: 9781.030433449172

    > 

e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\restricted_bwacs_model.py:445: RuntimeWarning: divide by zero encountered in true_divide
  self.np.divide(1, self.distances_matrix), self.beta)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\restricted_bwacs_model.py:447: RuntimeWarning: divide by zero encountered in true_divide
  self.np.divide(1, self.energies_matrix), self.gamma)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\restricted_bwacs_model.py:453: RuntimeWarning: invalid value encountered in multiply
  self.combination_matrix, _saving_energies_matrix)


• INICIANDO CLUSTERIZACIÓN: 

    > CLUSTER TYPE: KMEANS
    > ITERACIÓN 1
        - Centroides actuales: [array([57, 58]), array([17, 33]), array([10, 17]), array([61, 33]), array([ 7, 38])]
        - Nodos sin asignar: [ 7 30], [23 11]
        - Mejor costo anterior: inf
        - Costo total actual: 9739.5597891085

    > ITERACIÓN 2
        - Centroides actuales: [array([37., 69.]), array([29.11111111, 31.44444444]), array([22.77777778, 13.55555556]), array([54.4, 31.2]), array([16.8, 53.4])]
        - Nodos sin asignar: [32], [23]
        - Mejor costo anterior: 9739.5597891085
        - Costo total actual: 9469.008712103712

    > ITERACIÓN 3
        - Centroides actuales: [array([41.3, 59.2]), array([32.44444444, 33.77777778]), array([21.22222222, 14.88888889]), array([46., 10.]), array([20.4, 53.5])]
        - Nodos sin asignar: [35], [6]
        - Mejor costo anterior: 9469.008712103712
        - Costo total actual: 10376.00191633337

    > ITERACIÓN 4
        - Centroides act